<a href="https://colab.research.google.com/github/stavco9/datastreaming-final-project/blob/main/3.1-Deezy-Match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install DeezyMatch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 114.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB

In [9]:
import os
import tqdm
import time
import pandas as pd
from DeezyMatch import train as dm_train, plot_log, finetune as dm_finetune, inference as dm_inference, combine_vecs, candidate_ranker, candidate_ranker_init


In [5]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/IDC_MSc/Year1/DataStreamingAlgorithms/DatastreamingFinalProject"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/IDC_MSc/Year1/DataStreamingAlgorithms/DatastreamingFinalProject


In [10]:
words_set = []
for i in range(10):
  candidates_file = f"./Dataset/dataset_candidates_{i}.txt"
  with open(candidates_file, 'r') as f:
    words_list = f.read().split('\n')
    words_set.append({"words_list": words_list})

In [ ]:
for i in range(10):
  print(f"Dataset {i + 1} of 10")
  # generate vectors for candidates (specified in dataset_path)
  # using a model stored at pretrained_model_path and pretrained_vocab_path
  dm_inference(input_file_path="./DeezyMatchDataset/train_conf.yaml",
              dataset_path=f"./Dataset/dataset_candidates_{i}.txt",
              pretrained_model_path="./DeezyMatchDataset/models/wikigaz_en_model/wikigaz_en_model.model",
              pretrained_vocab_path="./DeezyMatchDataset/models/wikigaz_en_model/wikigaz_en_model.vocab",
              inference_mode="vect",
              scenario=f"./DeezyMatchDataset/candidates/test_{i}")
  print("========================")

In [ ]:
for i in range(10):
  print(f"Dataset {i + 1} of 10")
  # combine vectors stored in candidates/test and save them in combined/candidates_test
  combine_vecs(rnn_passes=['fwd', 'bwd'],
              input_scenario=f'./DeezyMatchDataset/candidates/test_{i}',
              output_scenario=f'./DeezyMatchDataset/combined/candidates_test_{i}',
              print_every=10)
  print("========================")

Streaming output truncated to the last 5000 lines.
0000330 candidates/test_2/embeddings/rnn_fwd_330
0000340 candidates/test_2/embeddings/rnn_fwd_340
0000350 candidates/test_2/embeddings/rnn_fwd_350
0000360 candidates/test_2/embeddings/rnn_fwd_360
0000370 candidates/test_2/embeddings/rnn_fwd_370
0000380 candidates/test_2/embeddings/rnn_fwd_380
0000390 candidates/test_2/embeddings/rnn_fwd_390
0000400 candidates/test_2/embeddings/rnn_fwd_400
0000410 candidates/test_2/embeddings/rnn_fwd_410
0000420 candidates/test_2/embeddings/rnn_fwd_420
0000430 candidates/test_2/embeddings/rnn_fwd_430
0000440 candidates/test_2/embeddings/rnn_fwd_440
0000450 candidates/test_2/embeddings/rnn_fwd_450
0000460 candidates/test_2/embeddings/rnn_fwd_460
0000470 candidates/test_2/embeddings/rnn_fwd_470
0000480 candidates/test_2/embeddings/rnn_fwd_480
0000490 candidates/test_2/embeddings/rnn_fwd_490
0000500 candidates/test_2/embeddings/rnn_fwd_500
0000510 candidates/test_2/embeddings/rnn_fwd_510
0000520 candidates

In [8]:
def format_word(word):
  if len(word) > 1:
    last_char = word[-1]
    while not last_char.isalpha():
      word = word[:-1]
      last_char = word[-1]

    first_char = word[0]
    while not first_char.isalpha():
      word = word[1:]
      first_char = word[0]

    word = word.lower()

  return word

In [12]:
#some_word_with_error = "appel"
some_sentance_with_error = "I wanna appel pleases or a bannana with a paer and a lemone, \
  or a wartermelon if possibe. I hope i don't have an errof in Londaon and in Jerusealm"
correct_sentence = "I wanna apple please or a banana with a pear and a lemon, \
  or a watermelon if possible. I hope i don't have an error in London and in Jerusalem"
diff_words_to_correct = list(set([format_word(x) for x in correct_sentence.split()]) - set([format_word(x) for x in some_sentance_with_error.split()]))
new_sentance = []
words_set_out = []

In [34]:
for index in range(10):
  # Ranking on-the-fly
  # find candidates from candidate_scenario
  # for queries specified by the `query` argument
  time_start_deezy = time.time()
  ranker = \
      candidate_ranker_init(candidate_scenario=f"./DeezyMatchDataset/combined/candidates_test_{index}",
                      query=[format_word(x) for x in some_sentance_with_error.split()],
                      ranking_metric="faiss",
                      selection_threshold=10.,
                      num_candidates=1,
                      search_size=100,
                      output_path=f"./DeezyMatchDataset/ranker_results/test_candidates_deezymatch_on_the_fly_{index}",
                      pretrained_model_path="./DeezyMatchDataset/models/wikigaz_en_model/wikigaz_en_model.model",
                      pretrained_vocab_path="./DeezyMatchDataset/models/wikigaz_en_model/wikigaz_en_model.vocab",
                      number_test_rows=len(some_sentance_with_error))

  ranker.rank()
  words_set[index]["deezy_match_df"] = ranker.output
  time_end_deezy = time.time()
  words_set[index]["time_deezy_sec"] = (time_end_deezy - time_start_deezy)

2024-03-29 18:42:12 147ee9725e13 [INFO] read input file: ./DeezyMatchDataset/combined/candidates_test_0/train_conf.yaml
2024-03-29 18:42:12 147ee9725e13 [INFO] pytorch will use: cuda
2024-03-29 18:42:13 147ee9725e13 [INFO] use a dataframe in test_tokenize.
2024-03-29 18:42:13 147ee9725e13 [INFO] number of labels, True: 0 and False: 30
2024-03-29 18:42:13 147ee9725e13 [INFO] skipping 0 lines


2024-03-29 18:42:13 147ee9725e13 [INFO] save test-data-class: tmp_f2fe89af-2d0c-4c8e-89fa-98f6e808eeb5/query/dataframe.df


  0%|          | 0/1 [00:00<?, ?it/s]



-- Combine vectors
Reading vectors from tmp_f2fe89af-2d0c-4c8e-89fa-98f6e808eeb5/query/embeddings/rnn_fwd*
0000000 tmp_f2fe89af-2d0c-4c8e-89fa-98f6e808eeb5/query/embeddings/rnn_fwd_0


-- Combine IDs

0000000 tmp_f2fe89af-2d0c-4c8e-89fa-98f6e808eeb5/query/embeddings/rnn_indxs_0


-- Combine vectors
Reading vectors from tmp_f2fe89af-2d0c-4c8e-89fa-98f6e808eeb5/query/embeddings/rnn_bwd*
0000000 tmp_f2fe89af-2d0c-4c8e-89fa-98f6e808eeb5/query/embeddings/rnn_bwd_0


-- Combine IDs

0000000 tmp_f2fe89af-2d0c-4c8e-89fa-98f6e808eeb5/query/embeddings/rnn_indxs_0
--- 5284.1745698452 seconds ---
Is faiss_id_candis already trained? True
=========== Start the search for 0 I
ID: 1/30 -- Number of found candidates so far: 3, searched: 100
=========== Start the search for 1 wanna
ID: 2/30 -- Number of found candidates so far: 5, searched: 100
=========== Start the search for 2 appel
ID: 3/30 -- Number of found candidates so far: 30, searched: 100
=========== Start the search for 3 pleases
ID: 4/30 -

2024-03-29 18:42:14 147ee9725e13 [INFO] save test-data-class: tmp_abeb7bf9-1479-4677-bb92-4356b1cc272b/query/dataframe.df


...

  0%|          | 0/1 [00:00<?, ?it/s]



-- Combine vectors
Reading vectors from tmp_d4c60303-8c51-4cc2-8d46-034a4465891a/query/embeddings/rnn_fwd*
0000000 tmp_d4c60303-8c51-4cc2-8d46-034a4465891a/query/embeddings/rnn_fwd_0


-- Combine IDs

0000000 tmp_d4c60303-8c51-4cc2-8d46-034a4465891a/query/embeddings/rnn_indxs_0


-- Combine vectors
Reading vectors from tmp_d4c60303-8c51-4cc2-8d46-034a4465891a/query/embeddings/rnn_bwd*
0000000 tmp_d4c60303-8c51-4cc2-8d46-034a4465891a/query/embeddings/rnn_bwd_0


-- Combine IDs

0000000 tmp_d4c60303-8c51-4cc2-8d46-034a4465891a/query/embeddings/rnn_indxs_0
--- 5296.655645608902 seconds ---
Is faiss_id_candis already trained? True
=========== Start the search for 0 I
ID: 1/30 -- Number of found candidates so far: 3, searched: 100
=========== Start the search for 1 wanna
ID: 2/30 -- Number of found candidates so far: 4, searched: 100
=========== Start the search for 2 appel
ID: 3/30 -- Number of found candidates so far: 29, searched: 100
=========== Start the search for 3 pleases
ID: 4/30

2024-03-29 18:42:27 147ee9725e13 [INFO] save test-data-class: tmp_e012c1ea-bc74-41b0-9652-a80d445ecd0e/query/dataframe.df


  0%|          | 0/1 [00:00<?, ?it/s]



-- Combine vectors
Reading vectors from tmp_e012c1ea-bc74-41b0-9652-a80d445ecd0e/query/embeddings/rnn_fwd*
0000000 tmp_e012c1ea-bc74-41b0-9652-a80d445ecd0e/query/embeddings/rnn_fwd_0


-- Combine IDs

0000000 tmp_e012c1ea-bc74-41b0-9652-a80d445ecd0e/query/embeddings/rnn_indxs_0


-- Combine vectors
Reading vectors from tmp_e012c1ea-bc74-41b0-9652-a80d445ecd0e/query/embeddings/rnn_bwd*
0000000 tmp_e012c1ea-bc74-41b0-9652-a80d445ecd0e/query/embeddings/rnn_bwd_0


-- Combine IDs

0000000 tmp_e012c1ea-bc74-41b0-9652-a80d445ecd0e/query/embeddings/rnn_indxs_0
--- 5298.231724500656 seconds ---
Is faiss_id_candis already trained? True
=========== Start the search for 0 I
ID: 1/30 -- Number of found candidates so far: 2, searched: 100
=========== Start the search for 1 wanna
ID: 2/30 -- Number of found candidates so far: 3, searched: 100
=========== Start the search for 2 appel
ID: 3/30 -- Number of found candidates so far: 33, searched: 100
=========== Start the search for 3 pleases
ID: 4/30

In [40]:
words_set_out = []
for index, words_list in enumerate(words_set):
    start = time.time()
    deezy_match_df = words_set[index]["deezy_match_df"]
    print(deezy_match_df)
    words_set_out.append({})
    words_set_out[index]["matched_words"] = []
    words_set_out[index]["match_distance"] = []
    #words_set_out[index]["words_frequency"] = []
    start = time.time()
    for word in some_sentance_with_error.split():
        word = format_word(word)
        if not word in words_list:
            print(f"Scanning {word} with Deezy Match")
            print("==========================")
            word_df_row = deezy_match_df.loc[deezy_match_df['query'] == word].iloc[0]
            match_word = format_word(list(word_df_row['faiss_distance'].keys())[0]) if bool(word_df_row['faiss_distance']) else None
            match_distance = list(word_df_row['faiss_distance'].values())[0] if bool(word_df_row['faiss_distance']) else -1
            words_set_out[index]["matched_words"].append(match_word)
            words_set_out[index]["match_distance"].append(match_distance)
            print(f"Potential match for word {word} is {match_word} with distance {match_distance}")
        else:
            print(f"Word {word} is correct. Skipping")
            words_set_out[index]["matched_words"].append(None)
            words_set_out[index]["match_distance"].append(None)
            #words_set_out[index]["words_frequency"].append(None)

    end = time.time()
    print(f"Total runtime of {(end - start)} seconds")
    words_set_out[index]["time_deezy_sec"] = words_list["time_deezy_sec"] + (end - start)

          query pred_score 1-pred_score         faiss_distance  \
id                                                               
0             I         {}           {}             {'I': 0.0}   
1         wanna         {}           {}         {'Wanna': 0.0}   
2         appel         {}           {}      {'papel': 4.7146}   
3       pleases         {}           {}    {'Settles': 5.0393}   
4            or         {}           {}            {'or': 0.0}   
5             a         {}           {}             {'a': 0.0}   
6       bannana         {}           {}   {'bandanna': 3.2076}   
7          with         {}           {}          {'with': 0.0}   
8             a         {}           {}             {'a': 0.0}   
9          paer         {}           {}     {'Parker': 3.4475}   
10          and         {}           {}           {'and': 0.0}   
11            a         {}           {}             {'a': 0.0}   
12       lemone         {}           {}     {'Lemon?:': 5.673}   
13        

In [41]:
OUT_DIR = './output/3-deezy-alg'
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)

In [42]:
print(words_set_out)

[{'matched_words': ['I', 'wanna', 'papel', 'settles', 'or', 'a', 'bandanna', 'with', 'a', 'parker', 'and', 'a', 'lemon', 'or', 'a', 'trawled', 'if', 'pose', 'I', 'hope', 'I', "don't", 'have', 'an', 'error', 'in', 'london', 'and', 'in', 'jerusalem'], 'match_distance': [0.0, 0.0, 4.7146, 5.0393, 0.0, 0.0, 3.2076, 0.0, 0.0, 3.4475, 0.0, 0.0, 5.673, 0.0, 0.0, 7.4863, 0.0, 7.2001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.2706, 0.0, 3.8871, 0.0, 0.0, 5.5064], 'time_deezy_sec': 1.6328842639923096}, {'matched_words': ['I', 'wanna', 'appealed', "stallion's", 'or', 'a', 'banana', 'with', 'a', 'parker', 'and', 'a', 'leone', 'or', 'a', 'warned', 'if', 'superimposed', 'I', 'hope', 'I', "don't", 'have', 'an', 'error', 'in', 'london', 'and', 'in', 'veljanovski'], 'match_distance': [0.0, 0.0, 6.1974, 4.9562, 0.0, 0.0, 3.6072, 0.0, 0.0, 3.4475, 0.0, 0.0, 4.8598, 0.0, 0.0, 7.5992, 0.0, 6.591, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.784, 0.0, 3.8871, 0.0, 0.0, 5.1666], 'time_deezy_sec': 1.6382646560668945}, {'matched_wor

In [43]:
for index, words_list in enumerate(words_set_out):
    df_comparisons = pd.DataFrame(data=words_list, index=[format_word(x) for x in correct_sentence.split()])
    #df_comparisons = df_comparisons.groupby(df_comparisons.index).first()
    df_comparisons.to_csv(f'{OUT_DIR}/dataset_{index}.csv')

In [44]:
df_comparisons

,matched_words,match_distance,time_deezy_sec
I,I,0.0000,1.625637
wanna,think,7.3751,1.625637
apple,padel,4.8308,1.625637
please,settles,5.0393,1.625637
or,or,0.0000,1.625637
a,a,0.0000,1.625637
banana,bandanna,3.2076,1.625637
with,with,0.0000,1.625637
a,a,0.0000,1.625637
pear,parker,3.4475,1.625637
